# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM
### - Допишите недостающие функции в MySVM (производные и обновление весов)

In [2]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([self.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return self.w * (1.0 / self.C)

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        if (1 - answer * self.f(x)) > 0:
            return -1.0
        else:
            return 0.0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            self.w -= step * (x * y * self.der_loss(x, y) + self.der_reg())
            
            # w_0 update
            self.w0 -= step * y * self.der_loss(x, y)

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

### C=10000

In [3]:
model_first = MySVM()
model_first.fit(X_train, y_train)
print(model_first.w, model_first.w0)

[-0.41072476  2.00505075  1.44069389  2.24268029  1.27190709  1.56290024
 -1.86694366 -0.33587805  0.04937335 -2.05935721 -0.92790213 -0.6932886
  1.5016486  -2.44691788 -1.62891772 -0.05627495  0.34800167  4.77037398
  2.3199871   0.31126646] -0.828969429811


In [4]:
predictions_first = model_first.predict(X_test)

In [5]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ..., 1 0 1] 2000 991


In [6]:
print(predictions_first, len(predictions_first), sum(predictions_first))

[0 1 0 ..., 1 1 1] 2000 1186


In [7]:
print(sum(predictions_first == y_test) / float(len(y_test)))

0.6085


### C=1

In [8]:
model_second = MySVM(C=1)
model_second.fit(X_train, y_train)
predictions_second = model_second.predict(X_test)

In [9]:
print(predictions_second, len(predictions_second), sum(predictions_second))

[0 1 1 ..., 1 1 1] 2000 1210


In [10]:
print(sum(predictions_second == y_test) / float(len(y_test)))

0.6325


### - Сравните качество с sklearn LinearSVC

### C=10000

In [11]:
from sklearn.svm import LinearSVC

In [12]:
model_LinearSVC_first = LinearSVC(penalty='l2', loss='hinge', max_iter=10000, C=10000).fit(X_train, y_train)
predictions_LinearSVC_first = model_LinearSVC_first.predict(X_test)

In [13]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ..., 1 0 1] 2000 991


In [14]:
print(predictions_LinearSVC_first, len(predictions_LinearSVC_first), sum(predictions_LinearSVC_first))

[1 0 1 ..., 1 0 0] 2000 1141


In [15]:
print(sum(predictions_LinearSVC_first == y_test) / float(len(y_test)))

0.692


### C=1

In [16]:
model_LinearSVC_second = LinearSVC(penalty='l2', loss='hinge', max_iter=10000, C=1).fit(X_train, y_train)
predictions_LinearSVC_second = model_LinearSVC_second.predict(X_test)

In [17]:
print(predictions_LinearSVC_second, len(predictions_LinearSVC_second), sum(predictions_LinearSVC_second))

[1 0 1 ..., 1 1 0] 2000 1022


In [18]:
print(sum(predictions_LinearSVC_second == y_test) / float(len(y_test)))

0.7975


## Выводы:

Установив одинаковые параметры для MySVM и LinearSVC, имеем:

1) C=10000 ---> accuracy_score(MySVM) < accuracy_score(LinearSVC)

2) C=1     ---> accuracy_score(MySVM) < accuracy_score(LinearSVC)